In [21]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

df_train = pd.read_csv('../../data/for_modelling/credit_score_train.csv')
df_val = pd.read_csv('../../data/for_modelling/credit_score_valid.csv')
df=pd.concat([df_train, df_val])
target='DEFAULT'
#df=df_train.copy()
df.head()

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,...,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE,DEFAULT
0,2783,1855,0,0.6665,0.00,0.0000,103,74,0.7184,0.0370,...,2.5003,0.0000,No,0,0,0,1,0,570,0
1,314430,445442,707468,1.4167,2.25,1.5882,35861,29157,0.8131,0.1141,...,0.5882,0.3704,High,1,0,1,1,0,691,0
2,161773,517674,2782496,3.2000,17.20,5.3750,3716,2533,0.6816,0.0230,...,0.3125,0.0581,No,1,1,1,1,1,520,0
3,16014,97685,20818,6.1000,1.30,0.2131,637,187,0.2936,0.0398,...,0.1639,0.7692,No,1,0,0,1,0,654,0
4,193225,1410542,2589215,7.3000,13.40,1.8356,5276,2325,0.4407,0.0273,...,0.1370,0.0746,No,1,1,0,1,1,552,0


In [22]:
# columns to remove from EDA
columns_to_remove = ["T_CLOTHING_12","T_ENTERTAINMENT_12", "T_GROCERIES_12", "T_GROCERIES_6", "T_HEALTH_12", "T_TAX_12", "T_TAX_6", "T_TRAVEL_12", "T_TRAVEL_6","T_UTILITIES_12", "T_UTILITIES_6", "T_EXPENDITURE_12", "T_EXPENDITURE_6"]
columns_to_remove

['T_CLOTHING_12',
 'T_ENTERTAINMENT_12',
 'T_GROCERIES_12',
 'T_GROCERIES_6',
 'T_HEALTH_12',
 'T_TAX_12',
 'T_TAX_6',
 'T_TRAVEL_12',
 'T_TRAVEL_6',
 'T_UTILITIES_12',
 'T_UTILITIES_6',
 'T_EXPENDITURE_12',
 'T_EXPENDITURE_6']

In [23]:

# map CAT_GAMBLING (No-0, Low-1, High-2)
df_train['CAT_GAMBLING'] = df_train['CAT_GAMBLING'].map({'No':0, 'Low':1, 'High':2})
df_val['CAT_GAMBLING'] = df_val['CAT_GAMBLING'].map({'No':0, 'Low':1, 'High':2})
# remove columns with high correlation
df_train.drop(columns=columns_to_remove, inplace=True)
df_val.drop(columns=columns_to_remove, inplace=True)
# remove outliers
from pyod.models.knn import KNN
clf = KNN(contamination=0.04)
rest_cols = [col for col in df_train.columns if col not in columns_to_remove]
# remove DEFAULT column
rest_cols.remove('DEFAULT')
clf.fit(df_train[rest_cols])
df_train['outliers'] = clf.labels_
# sum of outliers
df_train['outliers'].value_counts()

outliers
0    470
1     20
Name: count, dtype: int64

In [24]:
df_train = df_train[df_train['outliers'] == 0]
df_train.drop(columns='outliers', inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_train

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,R_CLOTHING_SAVINGS,...,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE,DEFAULT
0,2783,1855,0,0.6665,0.0000,0.0000,74,0.7184,0.0370,0.0555,...,2.5003,0.0000,0,0,0,0,1,0,570,0
1,314430,445442,707468,1.4167,2.2500,1.5882,29157,0.8131,0.1141,0.0805,...,0.5882,0.3704,2,1,0,1,1,0,691,0
2,161773,517674,2782496,3.2000,17.2000,5.3750,2533,0.6816,0.0230,0.0072,...,0.3125,0.0581,0,1,1,1,1,1,520,0
3,16014,97685,20818,6.1000,1.3000,0.2131,187,0.2936,0.0398,0.0065,...,0.1639,0.7692,0,1,0,0,1,0,654,0
4,193225,1410542,2589215,7.3000,13.4000,1.8356,2325,0.4407,0.0273,0.0037,...,0.1370,0.0746,0,1,1,0,1,1,552,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,20007,22508,457665,1.1250,22.8752,20.3334,861,0.5934,0.0725,0.0645,...,1.1111,0.0546,0,1,0,0,1,0,423,0
466,20846,268679,23162,12.8888,1.1111,0.0862,533,0.4094,0.0625,0.0048,...,0.0862,1.0000,1,1,0,0,1,0,584,0
467,89235,1106514,160623,12.4000,1.8000,0.1452,835,0.3460,0.0270,0.0022,...,0.0806,0.5556,0,1,0,0,1,0,631,0
468,197073,1021195,1863233,5.1818,9.4545,1.8246,3948,0.4691,0.0427,0.0082,...,0.1754,0.0962,2,1,0,1,1,1,572,0


In [25]:
X=df_train.drop(target, axis=1)
y=df_train[target]
X_val=df_val.drop(target, axis=1)
y_val=df_val[target]
# Box Cox and standart scalling
for col in rest_cols:
    X[col] = boxcox(X[col]+1)[0]
    X_val[col] = boxcox(X_val[col]+1)[0]
X[rest_cols] = scaler.fit_transform(X[rest_cols])
X_val[rest_cols] = scaler.transform(X_val[rest_cols])

In [77]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier

model1 = DecisionTreeClassifier(random_state=1)
model2 = KNeighborsClassifier()
model3 = LogisticRegression(random_state=1, max_iter=1000)
model4 = SVC(random_state=42, kernel='poly', degree=10)

estimators=[('DecisionTree', model1), ('KNN', model2), ('LR', model3)]

In [78]:
from sklearn.ensemble import VotingClassifier
model_hard = VotingClassifier(estimators=estimators, voting='hard')
model_hard.fit(X,y)

y_hat = model_hard.predict(X_val)
print('accuracy: ', recall_score(y_val, y_hat))

print('model.score: ', model_hard.score(X_val,y_val))

accuracy:  0.5897435897435898
model.score:  0.427536231884058


In [79]:
model_soft = VotingClassifier(estimators=estimators, voting='soft')
model_soft.fit(X, y)

y_hat = model_soft.predict(X_val)
recall_score(y_val, y_hat)

0.5897435897435898

In [88]:
from sklearn.metrics import classification_report
model_soft = VotingClassifier(estimators=estimators, voting='soft', weights=[0.05, 0.05, 0.90])
model_soft.fit(X, y)

y_hat = model_soft.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        99
           1       0.28      1.00      0.44        39

    accuracy                           0.28       138
   macro avg       0.14      0.50      0.22       138
weighted avg       0.08      0.28      0.12       138


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [90]:
from sklearn.ensemble import StackingClassifier
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X, y).score(X_val, y_val)

0.2826086956521739

In [98]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(n_estimators=10, random_state=0)
clf.fit(X, y)
y_hat = clf.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.83      0.29      0.43        99
           1       0.32      0.85      0.46        39

    accuracy                           0.45       138
   macro avg       0.57      0.57      0.45       138
weighted avg       0.68      0.45      0.44       138


In [99]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                  max_depth=2, # Maksymalna wysokość drzewa w słabym estymatorze
                                  min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                  max_features = 3, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                  random_state=0,
                                  n_jobs = -1)
model_rf.fit(X, y)
y_hat = model_rf.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.72      1.00      0.84        99
           1       0.00      0.00      0.00        39

    accuracy                           0.72       138
   macro avg       0.36      0.50      0.42       138
weighted avg       0.51      0.72      0.60       138


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [101]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=1)
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.31      0.43        99
           1       0.28      0.67      0.39        39

    accuracy                           0.41       138
   macro avg       0.49      0.49      0.41       138
weighted avg       0.58      0.41      0.42       138


In [102]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=1,
                                   learning_rate=0.01)
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.70      0.54      0.61        99
           1       0.26      0.41      0.32        39

    accuracy                           0.50       138
   macro avg       0.48      0.47      0.46       138
weighted avg       0.57      0.50      0.52       138


In [104]:
from xgboost import XGBClassifier
model=XGBClassifier(random_state=1,
                    learning_rate=0.01, # Szybkość "uczenia" się
                    booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                    max_depth=4 # Maksymalna głębokość drzewa 
                    )
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        99
           1       0.38      0.38      0.38        39

    accuracy                           0.65       138
   macro avg       0.57      0.57      0.57       138
weighted avg       0.65      0.65      0.65       138
